# Code for optimizing catalyst composition using active learning

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf


In [2]:
#np.random.seed(404)
np.set_printoptions(precision=2, suppress=True)

In [3]:
# Read data
data = pd.read_excel('Modelling_Data_Phase_1.xlsx', sheet_name ='Seed', index_col=False) #change the sheet name according to the tabs to run the various cycles in the current Phase
data.head()

,Zr,Cu,Co,Fe,STY
0,0.129,0.0,0.000,0.871,236.682003
1,0.135,0.0,0.185,0.680,294.040805
2,0.115,0.0,0.315,0.570,281.273559
3,0.117,0.0,0.460,0.423,243.321402
4,0.125,0.0,0.607,0.268,161.815333


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Zr      31 non-null     float64
 1   Cu      31 non-null     float64
 2   Co      31 non-null     float64
 3   Fe      31 non-null     float64
 4   STY     31 non-null     float64
dtypes: float64(5)
memory usage: 1.3 KB


In [5]:
# General stastistical data
data.describe()

,Zr,Cu,Co,Fe,STY
count,31.000000,31.00000,31.000000,31.000000,31.000000
mean,0.143742,0.24529,0.236065,0.374903,171.513337
std,0.097154,0.26235,0.257175,0.314445,100.103920
min,0.000000,0.00000,0.000000,0.000000,0.000000
25%,0.113500,0.00000,0.000000,0.000000,86.619788
50%,0.122000,0.20200,0.177000,0.423000,161.547789
75%,0.142000,0.41300,0.432000,0.658000,249.106089
max,0.438000,0.89100,0.851000,0.871000,324.248728


In [6]:
# Define X and Y from the data. Invert Y by multiplying with -1 to form a MAXIMIZATION problem as BO is MINIMIZATION by default

X = data[['Zr ', 'Cu ', 'Co ', 'Fe']].values
Y = -1*data['STY'].values.reshape(-1, 1) 

In [7]:
#Check the values of X or Y
Y

array([[-236.68],
       [-294.04],
       [-281.27],
       [-243.32],
       [-161.82],
       [ -99.86],
       [  -0.  ],
       [ -86.44],
       [-134.09],
       [-210.05],
       [-299.99],
       [-309.92],
       [  -0.  ],
       [ -90.26],
       [ -86.8 ],
       [ -91.53],
       [ -82.64],
       [ -83.37],
       [-324.25],
       [-218.9 ],
       [ -63.58],
       [-140.57],
       [ -71.58],
       [-161.55],
       [-239.7 ],
       [-238.42],
       [-254.89],
       [-304.16],
       [-321.49],
       [-121.7 ],
       [ -64.04]])

In [8]:
## Import necesarry libraries for Gaussian process regression 

from gpflow.models import GPR
from gpflow.models import SVGP
from gpflow.likelihoods import Gaussian
from gpflow.optimizers import Scipy
from gpflow.kernels import SquaredExponential as SE, Constant as C, White as W, SharedIndependent as SI
from gpflow.inducing_variables import SharedIndependentInducingVariables as SIIV, InducingPoints as IP
from sklearn.metrics import r2_score, mean_squared_error

c:\Users\Flo\anaconda3\envs\ramirez\lib\site-packages\gpflow\versions.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [9]:
#Define the kernels using squared exponential. The dimentions of lengthscales must match the number of input features
#This is a single objective task with 4 input features, where each feature correspond to the metal composition

kernel = SE(lengthscales=[0.1,1.3,0.1,0.3])

# Gaussian process regression
gp_model = GPR((X, Y), kernel=kernel)

# Optimize the lengthscales
opt = Scipy()
opt.minimize(gp_model.training_loss, gp_model.trainable_variables)


  message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
  success: True
   status: 0
      fun: 164.32687378043525
        x: [-1.771e+00 -2.898e-01  7.751e+00 -5.159e-01  1.184e+04
             5.952e+02]
      nit: 68
      jac: [-1.765e+00  2.725e+00  9.630e-03  2.894e+00 -1.889e-04
            -5.006e-04]
     nfev: 83
     njev: 83
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>

In [10]:
from sklearn.metrics import r2_score, mean_squared_error

# After optimization, you can make predictions with the trained model
Y_pred_mean, _ = gp_model.predict_y(X)  # Predicted mean values

# Evaluate R^2 score
r2 = r2_score(Y, Y_pred_mean)

# Evaluate mean squared error (RMSE)
mse = mean_squared_error(Y, Y_pred_mean)
rmse = np.sqrt(mse)

# Print the evaluation metrics with limited decimal places
print("R^2 score: {:.2f}".format(r2))
print("Root Mean Squared Error: {:.2f}".format(rmse))

R^2 score: 0.96
Root Mean Squared Error: 18.96


In [11]:
# Optimized kernel parameters

from gpflow.utilities import print_summary
gp_model.kernel.parameters


(<Parameter: name=softplus, dtype=float64, shape=[4], fn="softplus", numpy=array([0.16, 0.56, 7.75, 0.47])>,
 <Parameter: name=softplus, dtype=float64, shape=[], fn="softplus", numpy=11836.129769808527>)

In [12]:
# Check the optimized hyperparameters

optimized_lengthscales = gp_model.kernel.lengthscales.numpy()
print("Optimized Lengthscales:", optimized_lengthscales)

Optimized Lengthscales: [0.16 0.56 7.75 0.47]


# Bayesian optimization

In [13]:
from trieste.space import LinearConstraint
from trieste.space import Box

# Flo: To reproduce the first suggestions,  adjusts according to supplementary table 2:
Zr_lb = 0.0
Zr_ub = 0.20
Cu_lb = 0.0
Cu_ub = 1.00
Co_lb = 0.0
Co_ub = 1.00
Fe_lb = 0.0
Fe_ub = 1.00

const_lb = -10
const_ub = 10


# Define linear constraints. Apply lb and ub to the scalar product of the number vector and the feature vector 

constraints = [LinearConstraint(A=tf.constant
        ([[1, 1, 1, 1], [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]), 
        lb=tf.constant([1, Zr_lb, Cu_lb, Co_lb, Fe_lb]), 
        ub=tf.constant([1, Zr_ub, Cu_ub, Co_ub, Fe_ub]))]
constrained_search_space = Box([0, 0, 0, 0], [1, 1, 1, 1], constraints=constraints)


In [14]:
# Essential functions for formatting data

from trieste.data import Dataset

def observer(in_):
    in_ = tf.convert_to_tensor(in_)
    out_, _ = gp_model.predict_y(in_)
    out_ = tf.convert_to_tensor(out_)
    return Dataset(in_, out_)

def initial_data(in_, out_):
    in_ = tf.convert_to_tensor(in_)
    out_ = tf.convert_to_tensor(out_)
    return Dataset(in_, out_)

In [15]:
# Import necessary libraries to build model

from trieste.models.gpflow import GaussianProcessRegression
from trieste.bayesian_optimizer import BayesianOptimizer
from trieste.acquisition.rule import EfficientGlobalOptimization
from trieste.acquisition.function import Fantasizer
from trieste.acquisition import LocalPenalization
from trieste.acquisition.function import ExpectedHypervolumeImprovement
from trieste.acquisition.function import ExpectedImprovement
from trieste.acquisition.function import PredictiveVariance

#Fit the model
model = GaussianProcessRegression(gp_model, num_kernel_samples=10)


# Define acquisition functions. We use the ei rule for exploitation and pv rule for exploration

ei = ExpectedImprovement(constrained_search_space)
rule_ei = EfficientGlobalOptimization(builder=ei)

pv = PredictiveVariance()
rule_pv = EfficientGlobalOptimization(builder=pv)


# Bayesian optimizer
bo = BayesianOptimizer(observer, constrained_search_space)


In [16]:
# Run the Bayesian optimizer for single objective

batch_size = 30 # This number is user defined and determines the number of recommendation made by the BO. Typically 5-10 generations yield good results. For the project, we used a batch of 30. 

# Alternate between rule_ei or rule_pv as parmaters when runnig the BO for exploitation or exploration campaigns, respectively.
bo_result = bo.optimize(batch_size, initial_data(X, Y), model, rule_pv, track_state = False, fit_initial_model=False)

c:\Users\Flo\anaconda3\envs\ramirez\lib\site-packages\scipy\optimize\_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


Optimization completed without errors


In [17]:
# Get results from the Bayesian optimizer
bo_initial_data = bo_result.try_get_final_dataset()
bo_X = bo_result.try_get_final_dataset().query_points.numpy()[-batch_size:,:]

bo_Y = -1*bo_result.try_get_final_dataset().observations.numpy()[-batch_size:,:]
np.set_printoptions(precision=3, suppress=True)

result=(np.concatenate((bo_X, bo_Y), axis=1))


In [18]:
# Create dataframe with results 

dfresult = pd.DataFrame(result, columns = ['Zr','Cu','Co','Fe','STY'])
dfresult = dfresult.round(2)
dfresult

,Zr,Cu,Co,Fe,STY
0,0.2,0.80,0.00,0.00,73.51
1,0.2,0.00,0.80,0.00,36.25
2,0.2,0.00,0.51,0.29,202.03
3,0.2,0.33,0.23,0.24,226.42
4,0.2,0.80,0.00,0.00,64.51
5,0.2,0.20,0.60,0.00,109.75
6,0.2,0.57,0.23,0.00,124.84
7,0.2,0.51,0.00,0.29,170.65
8,0.2,0.00,0.80,0.00,35.70
9,0.2,0.00,0.42,0.38,246.14


In [19]:
#The above dataframe is the output of the current campiagn recommending catalyst composition worth investigation and its predicted yield.
#In the study we performed these experimental recommendation and measured the actual yield

# Comparision with synthesized compositions Phase 1, Cycle 1:

In [22]:
df_suggestions_paper = pd.read_excel('Full_catalytic_performance_data_Zenodo.xlsx', 
                                     sheet_name='Phase 1', 
                                     usecols='C:G', 
                                     skiprows=1,    
                                     nrows=6)
# Standardize column names by stripping whitespace
df_suggestions_paper.columns = df_suggestions_paper.columns.str.strip()
df_suggestions_paper.rename(columns={df_suggestions_paper.columns[-1]: 'STY'}, inplace=True)
df_suggestions_paper

,Zr,Cu,Co,Fe,STY
0,0.10,0.49,0.20,0.21,137.53
1,0.10,0.20,0.48,0.22,159.17
2,0.10,0.42,0.10,0.38,198.38
3,0.10,0.20,0.22,0.48,270.86
4,0.12,0.10,0.41,0.37,228.67
5,0.12,0.10,0.15,0.63,317.35


In [23]:
df_suggestions_repro = dfresult

from scipy.spatial.distance import euclidean

closest_matches = {}

for idx_paper, row_paper in df_suggestions_paper.iterrows():
    cols = ['Zr', 'Cu', 'Co', 'Fe']
    values_paper_ = df_suggestions_paper.loc[idx_paper].values[0:4] # ugly workaround, because row_paper[cols] and even row_paper['Fe'] FAILED for unknown reasons.

    distances = [ euclidean(values_paper_, row_repro.values[0:4]) for i_repro, row_repro in df_suggestions_repro.iterrows() ]

    closest_idx = np.argmin(distances)
    closest_matches[idx_paper] = {
        'i_repro': closest_idx,
        'distance': distances[closest_idx],
    }


closest_matches

{0: {'i_repro': 3, 'distance': 0.19339079605813714},
 1: {'i_repro': 29, 'distance': 0.15556349186104046},
 2: {'i_repro': 7, 'distance': 0.1902629759044045},
 3: {'i_repro': 11, 'distance': 0.20346989949375802},
 4: {'i_repro': 9, 'distance': 0.12884098726725127},
 5: {'i_repro': 25, 'distance': 0.16248076809271925}}

In [27]:
sorted_matches = sorted(closest_matches.items(), key=lambda x: x[1]['distance'])

for idx_paper, match_info in sorted_matches:

    distance = match_info['distance']
    
    print(f"Distance: {distance:.4f}")
    print()
    print(f"Paper:")
    print(df_suggestions_paper.loc[idx_paper].round(2))
    print()
    print(f"Repro:")
    print(df_suggestions_repro.loc[ match_info['i_repro'] ].round(2))
    print("-------------------------")


Distance: 0.1288

Paper:
Zr       0.12
Cu       0.10
Co       0.41
Fe       0.37
STY    228.67
Name: 4, dtype: float64

Repro:
Zr       0.20
Cu       0.00
Co       0.42
Fe       0.38
STY    246.14
Name: 9, dtype: float64
-------------------------
Distance: 0.1556

Paper:
Zr       0.10
Cu       0.20
Co       0.48
Fe       0.22
STY    159.17
Name: 1, dtype: float64

Repro:
Zr       0.00
Cu       0.15
Co       0.57
Fe       0.28
STY    120.10
Name: 29, dtype: float64
-------------------------
Distance: 0.1625

Paper:
Zr       0.12
Cu       0.10
Co       0.15
Fe       0.63
STY    317.35
Name: 5, dtype: float64

Repro:
Zr       0.20
Cu       0.00
Co       0.23
Fe       0.57
STY    294.55
Name: 25, dtype: float64
-------------------------
Distance: 0.1903

Paper:
Zr       0.10
Cu       0.42
Co       0.10
Fe       0.38
STY    198.38
Name: 2, dtype: float64

Repro:
Zr       0.20
Cu       0.51
Co       0.00
Fe       0.29
STY    170.65
Name: 7, dtype: float64
-------------------------
Distance: 